# Marketing Assistant

Copyright 2025, Denis Rothman

**Goal:** This notebook serves as the practical runtime for the **Strategic Marketing Engine**, as architected in Chapter 9. It demonstrates how the generic, multi-domain "Glass Box" Context Engine can be seamlessly repurposed to solve a variety of real-world marketing challenges without any changes to its core code.

This notebook will:
* Connect to the Pinecone knowledge base populated by the `Data_Ingestion_Marketing.ipynb` script.
* Use the generic Control Deck templates to interact with the engine.
* Execute the marketing use cases




# I. Inititalization

## GitHub

In [ ]:
# --- Downloading the utilities and the agents ---

# The !curl command is a simple and effective way to download raw files from a public GitHub repo.
# The -L flag ensures that it follows any redirects.

# print("Downloading helper files from public repository...")
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/helpers.py --output helpers.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/agents.py --output agents.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/registry.py.py --output registry.py.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine.py --output engine.py

# print("✅ Files downloaded successfully!")

In [ ]:
import requests
import os
from google.colab import userdata

def download_private_github_file(filename, destination_path="."):
    """
    Downloads a file from a private GitHub repository using a token
    stored in Colab Secrets.

    Requires a secret named 'GITHUB_TOKEN' to be set in the Colab UI.
    """
    # --- Configuration: Replace with your repository details ---
    owner = "Denis2054"
    repo = "Context-Engineering"
    branch = "main"
    # ---------------------------------------------------------

    try:
        # Securely fetch the GitHub token from Colab Secrets
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            raise userdata.SecretNotFoundError("Secret 'GITHUB_TOKEN' not found.")
    except userdata.SecretNotFoundError:
        print("🛑 Error: Secret 'GITHUB_TOKEN' not found.")
        print("Please add your GitHub Personal Access Token to the Colab Secrets Manager.")
        return
    except Exception as e:
        print(f"An error occurred while accessing secrets: {e}")
        return

    # Construct the GitHub API URL for the file
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{filename}?ref={branch}"

    # Prepare headers for authentication and to request the raw file content
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github.v3.raw"
    }

    try:
        # Make the request to the GitHub API
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # This will raise an HTTPError for bad responses (4xx or 5xx)

        # Determine the local filename
        local_filename = os.path.join(destination_path, os.path.basename(filename))

        # Save the file content locally
        with open(local_filename, "wb") as f:
            f.write(response.content)

        print(f"✅ Successfully downloaded '{filename}' to '{local_filename}'")

    except requests.exceptions.HTTPError as e:
        print(f"🛑 Error downloading '{filename}': {e}")
        if e.response.status_code == 404:
            print("   Please check that the owner, repo, file path, and branch are correct.")
        elif e.response.status_code == 401:
            print("   Authentication failed. Please check if your GITHUB_TOKEN is valid and has access to the repo.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- Example Usage: Download your utility files ---
# Ensure you have set 'GITHUB_TOKEN' in your Colab Secrets before running this.

download_private_github_file("commons/ch8/utils.py")
download_private_github_file("commons/ch8/helpers.py")
download_private_github_file("commons/ch8/agents.py")
download_private_github_file("commons/ch8/registry.py")
download_private_github_file("commons/ch8/engine.py")

✅ Successfully downloaded 'commons/ch8/utils.py' to './utils.py'
✅ Successfully downloaded 'commons/ch8/helpers.py' to './helpers.py'
✅ Successfully downloaded 'commons/ch8/agents.py' to './agents.py'
✅ Successfully downloaded 'commons/ch8/registry.py' to './registry.py'
✅ Successfully downloaded 'commons/ch8/engine.py' to './engine.py'


## Installation and client setup

In [ ]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [ ]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [ ]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active):
    """
    Runs the context engine, now with an optional, two-stage moderation check.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=result, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\\n\\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)

## Control Deck configuration

In [ ]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [ ]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the QuantumDrive"

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment': 4

Here’s the quick scoop on QuantumDrive Q-1:

- What it is: A pro-grade NVMe 2.0 SSD over PCIe Gen 5, aimed at video editors, 3D artists, and photographers (also appealing to PC builders/tech enthusiasts).
- Speeds: Up to 7,500 MB/s reads and 7,000 MB/s writes.
- Capacities: 2TB, 4TB, 8TB.
- Stays fast: Integrated graphene heat spreader to help prevent thermal throttling during long renders/transfers.
- Security/software: AES-256 hardware encryption plus the DataWeaver backup + encryption suite.
- Endurance: 3,000 TBW for the 4TB model.
- Warranty: 5-year limited.

What it means: It’s built to cut load and render times and keep performance steady for pro workflows, with encryption and backup baked in.

Caveats: Pricing isn’t listed, it’s currently on pre-order, and detailed specs like random I/O, latency, power, and endurance for the 2TB/8TB models aren’t provided.

Launch vibes: Campaign message is “End the Wait,” running on LinkedIn, Twitter, Instagram, and Reddit (#QuantumDrive #EndTheWait #BuiltForPros #SSD).

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 111.87 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'QuantumDrive overview\n'
                                       '- What it is: QuantumDrive Q-1 is a '
                                       'professional-grade solid-state drive '
                                       '(SSD) targeted at creative '
                                       'professionals (video editors, 3D '
                                       'artists, photographers).\n'
                                       '\n'
                                       'High-level mechanism\n'
                                       '- Interface/protocol: NVMe 2.0 over '
                                       'PCIe Gen 5.\n'
                                       '- Thermal design: Integrated graphene '
                                       'heat spreader designed to prevent '
                      

In [ ]:
#@title Product Marketing Copy Generation(Use Case 1)

# 1. Define the Goal: A research query that asks for a creative output.
goal = "Using the official product spec sheet, write a short marketing description for the new QuantumDrive Q-1. The description should be confident, aspirational, and focus on the benefits for creative professionals. Please cite your sources."


# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

# 1. Define the Goal: A research query focused on strategic analysis.
goal = "Analyze the ChronoTech press release and summarize their core product messaging and value proposition. Please cite your sources."


# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Meet the ChronoTech Chrono SSD Pro — “Speed for the Modern Creator.” It’s a flagship PCIe Gen 4 SSD built to kill workflow bottlenecks for digital artists and content creators. You get up to 7,300 MB/s sequential reads (and blazing writes), so projects load and transfer fast, and your timeline stays smooth. It’s on a proven PCIe Gen 4 platform, the 4TB model is priced competitively, and ChronoTech’s CEO says it delivers “the fastest possible read and write speeds.” If you want top-tier speed without blowing the budget, this is it. Available for purchase today.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 69.63 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Summarize information '
                                  'quickly and casually.", "style_guide": "Use '
                                  'informal language. Keep it brief and '
                                  'engaging. Imagine explaining it to a '
                                  'friend.", "instruction": "Summarize the '
                                  'provided facts using the casual style '
                                  'guide."}'},
    'planned_input': { 'intent_query': 'Executive brief of a press release '
                                       'focusing on core product messaging and '
                                       'value proposition; produce a short '
                                       'headline and a 2-3 sentence overview, '
                                     

In [ ]:
#@title Product Marketing Copy Generation(Use Case 2)


# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

## 1. Define the Goal: A research query that asks for a creative output.
goal = "Using the official product spec sheet, write a short marketing description for the new QuantumDrive Q-1. The description should be confident, aspirational, and focus on the benefits for creative professionals. Please cite your sources."



# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Definition
Project QuantumDrive is a PCIe Gen 5 NVMe 2.0 solid-state drive designed for creative professionals, including video editors, 3D artists, and photographers. It is offered in 2TB, 4TB, and 8TB capacities and includes pro-grade durability (3,000 TBW for the 4TB model) with a 5-year limited warranty. Security features include AES‑256 hardware encryption and the DataWeaver backup/encryption suite.

Function/Operation
- Performance: Delivers up to 7,500 MB/s sequential reads and 7,000 MB/s sequential writes to enable fast scrubbing, previews, and exports.
- Thermal management: An integrated graphene heat spreader prevents thermal throttling under sustained workloads.
- Security and data management: Hardware-based AES‑256 encryption with the DataWeaver suite for backup and encryption workflows.
- Interface: NVMe 2.0 over PCIe Gen 5. OS and device compatibility are not specified in the provided specifications.

Key Findings/Impact
- Workflow acceleration: Reduces wait times and accelerates timelines, renders, and asset loads for professional media production.
- Capacity for large projects: 2TB–8TB options accommodate extensive media libraries, caches, and proxy workflows.
- Sustained performance: Thermal design maintains high throughput during prolonged tasks.
- Reliability and support: Professional endurance (3,000 TBW for 4TB) and a 5‑year limited warranty enhance trust for production environments.
- Data protection: Hardware encryption and integrated backup/encryption tools help safeguard client assets.
- Documentation note: OS/device compatibility details are unspecified, and no public URL for the spec sheet was provided.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 155.30 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'Official manufacturer spec sheet '
                                       'located\n'
                                       '- Title: Product Specification Sheet: '
                                       'Project QuantumDrive — Product Name: '
                                       'QuantumDrive Q-1 (official '
                                       'manufacturer spec sheet). No public '
                                       'URL was provided in the supplied '
                                       'materials. [Source: Product '
                                       'Specification Sheet: Project '
                                       'QuantumDrive]\n'
                                       '\n'
                                       'Key specs and claims for creative '
                            

In [ ]:
#@title Writing a brand pitch recommendation(Use case 3)

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# =CONTROL DECK 3: A persuasive pitch ===
goal = "Write a persuasive pitch on our brand tone and voice guide"


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


Quick heads-up: there aren’t any actual facts to summarize—just a blank framework. What’s missing:

- Stakeholder pain points
- Desired outcomes
- Key value props and differentiation
- Proof points (stats, examples)
- Top objections + short responses
- Implementation plan (phases, owners, timeline)
- Clear call to action

Drop those in, and I’ll spin a tight, casual summary in no time.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 924.48 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Summarize information '
                                  'quickly and casually.", "style_guide": "Use '
                                  'informal language. Keep it brief and '
                                  'engaging. Imagine explaining it to a '
                                  'friend.", "instruction": "Summarize the '
                                  'provided facts using the casual style '
                                  'guide."}'},
    'planned_input': { 'intent_query': 'Persuasive executive pitch/one-pager '
                                       'advocating adoption and use of a Brand '
                                       'Tone & Voice Guide; confident, '
                                       'benefit-led narrative with clear '
                                       'str